In [1]:
import pandas as pd
from prophet import Prophet
import numpy as np
import pandas
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.vector_ar.vecm import *

In [2]:
data = pd.read_excel('SORA.xlsx')
df = pd.read_excel('SORA.xlsx')
data.reset_index(drop=True, inplace=True)

In [3]:
dates = data.drop(columns=["SORA", "EFFR"])

In [4]:
df = df.drop(columns=["SORA Publication Date"])

In [5]:
dates = dates.squeeze()

In [6]:
dates = dates.astype(str)

In [7]:
from statsmodels.tsa.base.datetools import dates_from_str
quarterly = dates_from_str(dates)

In [14]:
data = df[["SORA", "EFFR"]]
data.index = pandas.DatetimeIndex(quarterly)
data.dropna(inplace=True)

In [16]:
lag_order = select_order(data=data, maxlags=10, deterministic="ci", seasons=4)
lag_order.summary()

C:\Users\chris\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\chris\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\chris\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\chris\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(d

,AIC,BIC,FPE,HQIC
0,-9.246,-9.227,9.646e-05,-9.239
1,-9.260,-9.235,9.512e-05,-9.251
2,-9.289,-9.258,9.241e-05,-9.278
3,-9.334,-9.297,8.840e-05,-9.321
4,-9.338,-9.295,8.805e-05,-9.323
5,-9.338,-9.289,8.805e-05,-9.321
6,-9.342,-9.288,8.765e-05,-9.323
7,-9.357,-9.297,8.637e-05,-9.336
8,-9.364,-9.299*,8.576e-05,-9.341
9,-9.364,-9.293,8.578e-05,-9.339


In [17]:
lag_order.aic, lag_order.bic, lag_order.fpe, lag_order.hqic

(10, 8, 10, 10)

In [18]:
rank_test = select_coint_rank(data, 0, 3, method="trace", signif=0.1)
rank_test.rank

1

In [19]:
rank_test.summary()

r_0,r_1,test statistic,critical value
0,2,114.3,13.43
1,2,0.04084,2.705


In [20]:
model = VECM(data, deterministic="ci", seasons=4, k_ar_diff=lag_order.aic, coint_rank=rank_test.rank)

C:\Users\chris\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [21]:
vecm_res = model.fit()
vecm_res.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
season1,0.0174,0.006,3.095,0.002,0.006,0.028
season2,0.0002,0.006,0.038,0.969,-0.011,0.011
season3,0.0076,0.006,1.355,0.175,-0.003,0.019
L1.SORA,-0.0423,0.015,-2.753,0.006,-0.072,-0.012
L1.EFFR,0.0488,0.029,1.699,0.089,-0.008,0.105
L2.SORA,-0.0909,0.015,-5.907,0.000,-0.121,-0.061
L2.EFFR,0.1035,0.029,3.560,0.000,0.047,0.160
L3.SORA,-0.1397,0.015,-9.102,0.000,-0.170,-0.110
L3.EFFR,-0.0110,0.029,-0.375,0.707,-0.069,0.047
L4.SORA,-0.0679,0.015,-4.418,0.000,-0.098,-0.038


In [52]:
pred = vecm_res.predict(steps=365, alpha=0.05)

In [53]:
pred

(array([[3.52371264, 5.33733808],
        [3.48305076, 5.34312996],
        [3.43581458, 5.35169646],
        [3.40671274, 5.36050265],
        [3.42943936, 5.36546881],
        [3.43510086, 5.36408171],
        [3.41911767, 5.37248336],
        [3.38991335, 5.38069171],
        [3.40020476, 5.38338956],
        [3.39998621, 5.38485418],
        [3.4009094 , 5.39134077],
        [3.38227811, 5.39620715],
        [3.38007467, 5.3997372 ],
        [3.36898702, 5.40148134],
        [3.3692656 , 5.4079753 ],
        [3.35810114, 5.41235569],
        [3.36211332, 5.41442159],
        [3.34961653, 5.41542702],
        [3.34677082, 5.42213397],
        [3.33644798, 5.42620587],
        [3.34408131, 5.4277169 ],
        [3.33342056, 5.42826049],
        [3.32933136, 5.43457762],
        [3.31744733, 5.43847942],
        [3.3255638 , 5.43972474],
        [3.31679746, 5.43978864],
        [3.31386117, 5.44570913],
        [3.30194023, 5.44930593],
        [3.30987478, 5.45026325],
        [3.301

In [47]:
pred0 = pred[0]
pred1 = pred[1]
pred2 = pred[2]

In [48]:
df0 = pd.DataFrame(pred0, columns =['SORA', 'EFFR'])
df1 = pd.DataFrame(pred1, columns =['SORA', 'EFFR'])
df2 = pd.DataFrame(pred2, columns =['SORA', 'EFFR'])

In [43]:
df_merged = pd.concat([df0, df1])
df_merged = pd.concat([df_merged, df2])

In [44]:
df_merged

,SORA,EFFR
0,3.523713,5.337338
1,3.483051,5.343130
2,3.435815,5.351696
3,3.406713,5.360503
4,3.429439,5.365469
...,...,...
360,4.603293,7.490333
361,4.599521,7.490598
362,4.602209,7.496719
363,4.595728,7.500570
